In [3]:
data = pd.read_csv("btc_tx_2011_2013.csv") #13- 14 - 15 e bakalim corr var mi hangi featurelar arasi average shortest path length etc.
# flow bazli metrikler
# her biri icin graph olusturulup time seriese bagli graphlar
# makalelerdeki featurelar (stanford 2015 | Group work) aynilarini deneriz
# 100k nodes randomly sampled, multiple times. Per month and closure price, price changing return % 
# pearson spearman corrs

In [4]:
data.head()

,0437cd7f8525ceed2324359c2d0ba26006d92d856a9c20fa0241106ee5a597c9,f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16,2009-01-09 03:54:39,50.0
0,f4184fc596403b9d638783cf57adfe4c75c605f6356fbc...,ea44e97271691990157559d0bdd9959e02790c34db6c00...,2009-01-12 03:30:25,10.0
1,f4184fc596403b9d638783cf57adfe4c75c605f6356fbc...,a16f3ce4dd5deb92d98ef5cf8afeaf0775ebca408f708b...,2009-01-12 03:30:25,40.0
2,a16f3ce4dd5deb92d98ef5cf8afeaf0775ebca408f708b...,591e91f809d716912ca1d4a9295e70c3e78bab077683f7...,2009-01-12 06:02:13,30.0
3,591e91f809d716912ca1d4a9295e70c3e78bab077683f7...,298ca2045d174f8a158961806ffc4ef96fad02d71a6b84...,2009-01-12 06:12:16,1.0
4,591e91f809d716912ca1d4a9295e70c3e78bab077683f7...,12b5633bad1f9c167d523ad1aa1947b2732a865bf5414e...,2009-01-12 06:12:16,29.0


In [5]:
data.columns = ["tx_hash_from","tx_hash_to","datetime","amount_bitcoins"]

In [6]:
data

,tx_hash_from,tx_hash_to,datetime,amount_bitcoins
0,f4184fc596403b9d638783cf57adfe4c75c605f6356fbc...,ea44e97271691990157559d0bdd9959e02790c34db6c00...,2009-01-12 03:30:25,10.000000
1,f4184fc596403b9d638783cf57adfe4c75c605f6356fbc...,a16f3ce4dd5deb92d98ef5cf8afeaf0775ebca408f708b...,2009-01-12 03:30:25,40.000000
2,a16f3ce4dd5deb92d98ef5cf8afeaf0775ebca408f708b...,591e91f809d716912ca1d4a9295e70c3e78bab077683f7...,2009-01-12 06:02:13,30.000000
3,591e91f809d716912ca1d4a9295e70c3e78bab077683f7...,298ca2045d174f8a158961806ffc4ef96fad02d71a6b84...,2009-01-12 06:12:16,1.000000
4,591e91f809d716912ca1d4a9295e70c3e78bab077683f7...,12b5633bad1f9c167d523ad1aa1947b2732a865bf5414e...,2009-01-12 06:12:16,29.000000
...,...,...,...,...
66199067,7ebd2c87efead031e8b6aeecc1ef4032551d9580726a1f...,5c6684d182093e8a8cbf8434f6dbb6c0a69684885b5c22...,2014-01-02 14:59:25,0.610000
66199068,6556332ea7196ac64e29d78576a7a6cdc4cbfecbd29f47...,653cab2118c19528ef711d45e5af39971f00e307da3ef4...,2014-01-02 14:59:25,0.026032
66199069,ddce84f4e7da169c0d470cd8c090e2c6be862d458b4f1c...,653cab2118c19528ef711d45e5af39971f00e307da3ef4...,2014-01-02 14:59:25,0.009715
66199070,e649a0bc622c6c499e81f85bae2ba95b588a7148b84509...,86985d45d89f3ad367cbbb96b3a1b2119ed8ed103be124...,2014-01-02 14:59:25,0.023515


In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.shape

In [8]:
data["datetime"]=pd.to_datetime(data["datetime"])
data["datetime"] = data["datetime"].dt.date

KeyError: 'datetime'

In [ ]:
data

In [ ]:
df = data.sort_values(by= ["datetime"])

In [ ]:
df.to_csv('Sorted.csv')

In [3]:
df = pd.read_csv("Sorted.csv")

In [4]:
df["datetime"]=pd.to_datetime(df["datetime"]).dt.date

In [ ]:
sliced = slice_by_weeks(df)

In [ ]:
with open('slices.pkl', 'wb') as f:
    pickle.dump(sliced, f)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from datetime import datetime
import gc
import pickle

In [2]:
def slice_by_weeks(df):
    groups = []
    prev=0
    end_of_week1 = False
    end_of_week2 = False
    for i in range(df.shape[0]):
        if df["datetime"].iloc[i].weekday() == 6:
            end_of_week1 = True
        elif df["datetime"].iloc[i].weekday() == 0:
            end_of_week2 = True
        if end_of_week1 and end_of_week2:
            #print(prev, i)
            groups.append(df.iloc[prev:i])
            prev = i
            end_of_week1 = False
            end_of_week2 = False
    return groups

def createGraph(data): # directed graph
    G=nx.from_pandas_edgelist(data,source="tx_hash_from",target ="tx_hash_to", edge_attr="amount_bitcoins", create_using=nx.DiGraph())
    return G

In [2]:
with open('slices.pkl', 'rb') as f:
    sliced = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'slices.pkl'

In [4]:
len(sliced)

519

In [5]:
graphs = []
index =470
for i in sliced[470:519]:
    graphs.append(createGraph(i))
    index+=1
    print(index, end="\r")

In [6]:
with open('graphs470-519.pkl', 'wb') as f:
    pickle.dump(graphs, f)